导入所需要的库（llama_cpp库如果直接安装失败可以尝试下载对应的whl包进行安装，下载地址：https://abetlen.github.io/llama-cpp-python/whl/cpu/llama-cpp-python/ ）

In [ ]:
from llama_cpp import Llama
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

本地部署大模型：需在huggingface上下载模型文件，下载地址：https://huggingface.co/Triangle104/Qwen2.5-3B-Instruct-abliterated-Q4_K_M-GGUF/tree/main

In [2]:
llm = Llama(model_path="D:\model\qwen2.5-3b-instruct-q4_k_m\qwen2.5-3b-instruct-abliterated-q4_k_m.gguf", verbose=False,)

llama_new_context_with_model: n_ctx_per_seq (512) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


一个模型对话的示例：

In [5]:
prompt = "Where is Southeast University located?"
output = llm.create_chat_completion(
    messages=[{
        "role": "user",
        "content": prompt
    }],
    max_tokens=200,
    stream=False,
)
answer = output["choices"][0]["message"]["content"]
print(answer)

Southeast University is located in Nanjing, Jiangsu Province, China.


请比较一下提示词工程对大模型输出的影响,
下面是第三次实验的机器翻译任务（可以通过对translate函数进行修改加入一些提示词工程），也可以尝试其他任务

In [6]:
def translate(english_sentence):
    instruntion = "please translate the following english sentence to chinese"

    output = llm.create_chat_completion(
        messages=[{
            "role": "user",
            "content": instruntion+" "+english_sentence
        }],
        max_tokens=200,
        stream=False,
    )
    answer = output["choices"][0]["message"]["content"]
    return answer

In [ ]:
chinese_dev_list = []
english_dev_list = []
with open("D:/file/file/人工智能概论/code/transformer_trans/data/dev.txt","r") as f:
    for line in f.readlines():
        english,chinese = line.strip().split("\t")
        chinese_dev_list.append(chinese)
        english_dev_list.append(english)

references = []
candidates = []
for i in range(10):
    e_s = english_dev_list[i]
    c_s = chinese_dev_list[i]
    answer = translate(e_s)
    references.append([c_s])
    candidates.append(answer)
    print('reference:', c_s)
    print('candidate:', answer)
    
smoothie = SmoothingFunction().method1
corpus_score = corpus_bleu(
    references,
    candidates,
    smoothing_function=smoothie
)
print(corpus_score)

reference 照顾好自己。
candidate Take care. 可以翻译成中文是 "保重。"
reference 在这等着。
candidate Wait here. 的中文翻译是 "等在这里。"
reference 做得好！
candidate 非常好！
reference 他向我施压。
candidate 他壓著我。
reference 他努力學習。
candidate 他学习很用功。
reference 他企图说谎。
candidate 他喜欢说谎。
reference 他很容易觉得累。
candidate 他容易累。
reference 他很老。
candidate 他非常老。
reference 你有电视吗？
candidate Do you have a TV? 的中文翻译是 "您有一台电视吗？"
reference 你們有小孩嗎?
candidate Do you have kids? 的中文翻译是 "您有孩子吗？"
0.041374412020518815
